In [ ]:
!pip install spacy-transformers
#modelo para la comparación de palabras
!python -m spacy download en_core_web_lg 
#libreía de extracción pke
!pip install git+https://github.com/boudinfl/pke.git
!python -m nltk.downloader stopwords
!python -m nltk.downloader universal_tagset
!python -m spacy download en 

In [ ]:
# Frameworks
import numpy as np
import pandas as pd
from google.colab import drive
from bs4 import BeautifulSoup as bs
import requests
import copy
import re
#importacion de librerias a utilizar
import nltk #utilizada para las stopwords
nltk.download('punkt')
import pke
from pke.lang import stopwords
import spacy #vectorizacion y similitud de las palabras
import en_core_web_sm
import spacy_transformers
import en_core_web_lg
#librerías para la tokenización y el limpiado de los datos
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
# librerias para autorizar escribir un sheet de drive
from google.colab import auth
import gspread
from google.auth import default

In [ ]:
# Initializacion google drive
drive.mount('/content/drive')

# Inicializacion del modelo "en_core_web_lg" de Spacy
nlp = en_core_web_lg.load()

# Linea para añadir stopwords a la lista que se tiene por default
all_stopwords_gensim = STOPWORDS.union(set(['var', 'anticipates','font' ,'datalayer','dataLayer','emoji','domcontentloaded','attachEvent','gtm4wp','wp','gtm','array','flag', 'n', 't','contactemoji','wpemoji','twemoji','important','year','jquery','rev'])) #Añadir palabras a stopwords list default

In [4]:
# ------------------------INICIO DE EJECUCION EXTRACCION DE EXPRESIONES REGULARES--------------



# ---------------Optional-----------------------------
# Add field 'ID_text' for reading all files text saved.
# url_len = len(df_main)
# list_text = [str(i)+'.txt' for i in range(url_len)]

# df_main["id_txt"] = list_text
# df_main.head(2)

In [5]:
# Function to read all urls and save the text content as text file format
def get_urls(df):
    df2 = df[["Url"]]
    cont=0
    names = []
    for url in df2["Url"]:
        
        r = requests.get(url)
        soup = bs(r.text, 'lxml')
        
        name_txt= str(cont)+'.txt'
        names.append(name_txt)
        cont = cont + 1
        
        f = open('/content/drive/MyDrive/Colab Notebooks/file_txt/'+name_txt, "w") 
        f.write(soup.text)
        f.close()
    
    df2['id_txt'] = names       
    return df2

In [6]:
# Function to get the reference id by each link
def get_opportunity_id(list_id, file):
    try:
        list_id.append(re.findall('(SFOP[0-9]*|SFOP-[0-9]*|SFOP-[a-zA-Z]*|SFOP[a-zA-Z]*)', file)[0])
    except:
        list_id.append("")

    return list_id

In [7]:
# Function that receives a slice text and compare if corresponding to some month
def has_a_month(item):
  months = ['january', 'february', 'march','april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
  for month in months:
    if month in item:
      return True
  return False

# Function to clean text and define feature format
def extract_date(row):
  for row_entry in row:
    row_as_list = row_entry.replace(':','\n').split('\n')
    for item in row_as_list:
      if has_a_month(item.lower()):
        word=item.lower().strip().replace('\xa0',' ').replace(",","").replace("(","").replace(")","").replace("\xe2","").replace(" at 11","")
        word=re.sub('th ',' ',word)
        word=re.sub('59 pm e(s)?(d)?t (on)?',' ',word)
        return pd.to_datetime(item.strip(), errors='ignore'),pd.to_datetime(word.strip(), errors='ignore')
  return ''

In [8]:
# Function to get the opening date of each announcement
def get_date_open(list_date_open, file):    
    try:
        exp=re.findall('([\n]Announcement issuance date:.+([\xa0]|[\n]))|([\n]Funding Opportunity Announcement(\nBureau of Democracy, Human Rights, and Labor\n)?.+([\xa0]|[\n]))',file)[0]
        list_date_open.append(extract_date(exp)[0])
    except:
        list_date_open.append("")

    return list_date_open

In [9]:
# Function to get the closing date of each announcement
def get_date_close(list_date_close, file):
    try: 
        exp=re.findall('([\n](Proposal )?[Ss]ubmission [Dd]eadline.+([\xa0]|[\n]))|([\n]Application [Dd]eadline.+([\xa0]|[\n]))',file)[0]
        list_date_close.append(extract_date(exp)[1]) 
    except:
        list_date_close.append("")
   
    return list_date_close

In [10]:
# Define if contains the sign
def has_a_sign(item):
  if "$" in item:
      return True
  return False

# Create list with slice text that contains some value or amount
def extract_amount(row_list):
  amts_found = []
  for tuple_in_row in row_list:
    for string_in_tuple in tuple_in_row:
      if(has_a_sign(string_in_tuple)):
        string_in_tuple_2=string_in_tuple.split('$')
        for elemt in string_in_tuple_2:
          amts_found.append(''.join(re.findall('[0-9,]',elemt)))
  if amts_found[0]=='' and amts_found[1]!='':
    amts_found[0]=amts_found[1]
  elif amts_found[0]=='' and amts_found[1]=='' and amts_found[2]!='':
    amts_found[0]=amts_found[2]
  elif amts_found[0]=='' and amts_found[1]=='' and amts_found[2]==''and amts_found[3]!='':
    amts_found[0]=amts_found[3]
  else:
    amts_found[0]
    
  return float(amts_found[0].replace(",",""))

In [11]:
# Function to get the minimium value of announcement
def get_amount_min(amount, file):
    try:
        exp=re.findall('([\n]Total [Ff]unding [Ff]loor.+[\n])|([\n][Ff]unding [Ff]loor.+[\n])|([\n][Ff]unding [Ll]imit.+[\n])',file)
        amount.append(extract_amount(exp))
    except:
        amount.append("")
    
    return amount

In [12]:
# Function to get the high value of announcement
def get_amount_max(amount, file):
    try:
        exp=re.findall('([\n][Ff]unding [Cc]eiling.+[\n])|([\xa0 ][Ff]unding [Cc]eiling.+[\n])|([\n][Tt]otal [Ff]unding [Cc]eiling.+[\n])|(and be no more than.+[\n])',file)
        amount.append(extract_amount(exp))
    except:
        amount.append("")
    
    return amount

In [13]:
# Validate if slice text does match with keyword list 'contr'
def has_a_cont(item):
  contr = ['Open', 'Cooperative', 'concept notes', 'Concept notes']
  for month in contr:
    if month in item:
      return True
  return False

def extract_contr(row_list):
  conts_found = []
  for tuple_in_row in row_list:
    for string_in_tuple in tuple_in_row:
      row_as_list = string_in_tuple.replace(':','\n').split('\n')
      for item in row_as_list:
        if has_a_cont(item):
          return item.strip()
  return ''

In [14]:
# Function to get announcement contribution type from regular expresion
def get_contribution_type_id(contribution, file):

    try:
        exp=re.findall('([\n][Aa]nnouncement [tT]ype.+[\n])|([\n][Tt]ype of [Ss]olicitation.+[\n])|([\n][Cc]oncept [Nn]otes.+[\n])',file)
        contribution.append(extract_contr(exp))
    except:
        contribution.append("")
    
    return contribution

In [15]:
# Function to clean text of description
def extract_desc(row):
  desc_found = []
  for rows in row:
    row_as_list = rows.replace('\n','').split('\n')
    return row_as_list[0]
  return ''

In [16]:
# Function to get announcement description from regular expresion
def get_description(description, file):
    try:
        exp=re.findall('(([\n]FY.+[\n]*)|([\n]DRL.+[\n]))',file)[0]
        description.append(extract_desc(exp))
    except:
        description.append("")
        
    return description

In [17]:
# Function extracts the region of each title in the announcement, from a preloaded list of countries
def extract_region(list_region_global, file, data_into_list):
  region_section = []
  list_region = []
  region_section.append(re.split('-', file)[0])

  for i in data_into_list:
    if re.findall(i, region_section[0]):
      list_region.append(i)

  list_to_string = ', '.join(list_region)
  list_region_global.append(list_to_string)
  
  return list_region_global

In [18]:
# Main function to get all words scrapyed of text and generate dataframe output
def main_regex(df):
  df2 = df[["Url","id_txt"]]
  id_cov = []
  date_open =[]
  date_close =[]
  amount_min =[]
  amount_max =[]
  contri_type =[]
  desc = []
  list_region_global = []
  
  # Read countries data
  file_countries = open('/content/drive/MyDrive/Colab Notebooks/all_countries.txt', 'r')
  data = file_countries.read()
  data_into_list = data.split("\n")

  # Initialization cicle for extract variables with regex
  for k in df2["id_txt"]:
    file = open('/content/drive/MyDrive/Colab Notebooks/file_txt/'+k, "r")
    document = file.read()
    get_opportunity_id(id_cov, document)
    get_date_open(date_open, document)
    get_date_close(date_close, document)
    get_amount_min(amount_min, document)
    get_amount_max(amount_max, document)
    get_contribution_type_id(contri_type, document)
    get_description(desc, document)
    extract_region(list_region_global, document, data_into_list)
    file.close()

    
  df2["id_cov"] = id_cov
  df2["date_open"] = date_open
  df2["date_close"] = date_close
  df2["amount_min"] = amount_min
  df2["amount_max"] = amount_max
  df2["contribution_type"] = contri_type
  df2["description"] = desc
  df2["region"] = list_region_global

  return df2

In [19]:
# --------------------INICIALIZACION EXTRACCION DE PALABRAS MEDIANTE PKE Y SPACY--------------

In [20]:
def extraccion_pke(document_txt, list_crudo):
  pos = {'NOUN', 'PROPN', 'ADJ', 'ADV'}

  #Remover caracteres especiales de los textos y convertir el texto en lowercase
  texto_prueba = document_txt.strip().lower()
  #Variable para limpiar el texto de caracteres especiales 
  CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 
  texto_prueba = re.sub(CLEANR, '', texto_prueba)
  #Quitar oraciones que se tengan entre brackets con la expresion regular \{.*?\}
  texto_prueba = re.sub("\{.*?\}", "", texto_prueba) 
  # Tokenizacion de las palabras y eliminación de las palabras claves previamente establecidas.
  texto_prueba_token = word_tokenize(texto_prueba)
  texto_prueba_sin_sw = [word for word in texto_prueba_token if not word in all_stopwords_gensim]
  filtered_text = " ".join(texto_prueba_sin_sw)
  filtered_text

  #Detalle PKE
  extractor = pke.unsupervised.TopicRank()
  extractor.load_document(input=filtered_text, language='en') # loading a document populates the extractor.sentences list (DETALLE PKE)
  extractor.candidate_selection(pos=pos)
  extractor.candidate_weighting()
  keyphrases = extractor.get_n_best(n=4, stemming=False)
  
  lista_keyword = [] #Lista de palabras claves extraidas con PKE
  
  for word,number in keyphrases:
      lista_keyword.append(word) #Creacion lista de keywords
      #lista_keyword + word + ","
      
  list_crudo.append((filtered_text,(lista_keyword)))

  return list_crudo

In [21]:
def main_model(df_model):
  
    list_crudo = []
    

    for k in df_model["id_txt"]:
        file = open('/content/drive/MyDrive/Colab Notebooks/file_txt/'+k, "r")
        document = file.read()
        extraccion_pke(document, list_crudo)
        file.close()
    
    return list_crudo

def create_df1(list_crudo):
  df1 = pd.DataFrame(list_crudo) #Transformar la lista del output en un dataframe
  df1 = df1.rename(columns={0:'file_text', 1: 'palabras_clave_pke'}) #renombrar columnas

  return df1




In [22]:
output_principal = [] # Output principal donde se tiene URL, palabras claves con PKE y listas de 5 principales match con la similitud de spacy

# count = 0 --> id_text

def similitud(df1, df_ods,df_main):

    for element,count in zip(df1.palabras_clave_pke,df_main.id_txt):
       
        lista_similarity_ods = [] #Lista donde se encuentran los 5 mayores resultados cuando se realizo el match de las keywords con el diccionario de ODS
        palabra_clave = element #Keyword de PKE

        for word in  palabra_clave:
            for row, row2 in zip(df_ods['ODS'],df_ods['Keyword']):
                ODS = row #Numero de la ODS
                ODS_kw = row2 # Palabra relacionada a la ODS
                doc3 = nlp(word) #Vectorizacion de la keyword de PKE
                doc4 = nlp(ODS_kw) # Vectorizacion de la palabra relacionada de la ODS
                similarity = round(doc3.similarity(doc4),5) 
                #lista_similarity_ods.append((word, ODS_kw, ODS, similarity)) #Lista con: keyword PKE, palabra clave PKE, Numero de la ODS, resultado de la similitud con spacy
                output_principal.append((word, ODS_kw, ODS, similarity,count)) #Creacion de output principal

    return output_principal
    

In [ ]:
if __name__ == '__main__':
  # Read URL list from .csv and the file text content then concat both and denomined 'df_main'
  
  # Inicio de main regex

  df_main = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/url_list.csv')
  
  df_total = get_urls(df_main)

  df_regex = main_regex(df_total)

  # Inicio de main model
  
  list_crudo_ = main_model(df_total)
  
  df_1 = create_df1(list_crudo_)

  # carga de dataframe ODS
  df_ods = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ODS.csv', sep=',') 

  output = similitud(df_1, df_ods, df_total)

  df2 = pd.DataFrame(output) #Transformar la lista del output en un dataframe
  df2 = df2.rename(columns={0: 'palabras_clave_pke',1: 'keyword',2:'ODS',3:'similarity',4:'id'}) #renombrar columnas

  # Obtencion de maximo en similitudes
  ks = df2.groupby(['id'],sort=False)['similarity'].max()
  ks.reset_index()

  result = pd.merge(df2, ks,how="inner", on=["id", "similarity"])
  result.drop_duplicates('id')

  # Concatenacion de Dataframes Regex y Modelo
  df_regex['keyword'] = result['keyword']
  df_regex['ods'] = result['ODS']
  df_regex['similarity_percentage'] = result["similarity"]*100
  df_regex.head(2)

  # Exportacion de Dataframe final a .csv cargada en Drive
  
  url = '/content/drive/MyDrive/Colab Notebooks/data_loaded.csv'
  load_csv_drive = df_regex.to_csv(url, index=False)

    # Exportacion de Dataframe final a sheet 
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)


  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)


  spreadsheet_key = '1OWfmUpYvOqg9f_qDP2StgHUsLwg0wOBkxpLjjaP6Yls'
  workbook = gc.open_by_key(spreadsheet_key)

  workbook.values_update(
    'Hoja 1!A1',
    params={
        'valueInputOption': 'USER_ENTERED'
    },
    body={
        'values': [df_regex.columns.values.tolist()] + df_regex.values.tolist()
    }
  )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
